In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
def cv_show(img, name='image'):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

## 圖像金字塔
* 高斯金字塔
* 拉普拉斯金字塔

### 高斯金字塔
- 向下採樣(縮小)
    - 與高斯內核卷積
    - 去除偶數行和列
- 向上採樣(放大)
    - 每個方向擴大兩倍, 新增的行列用0填充
    - 使用相同的內核與放大後的圖像做卷積, 獲得近似值

In [3]:
path = 'dataset/AM.png'

In [4]:
img = cv2.imread(path)
cv_show(img)
print(img.shape)

(442, 340, 3)


In [5]:
up = cv2.pyrUp(img)
cv_show(up)
print(up.shape)

(884, 680, 3)


In [6]:
down = cv2.pyrDown(img)
cv_show(down)
print(down.shape)

(221, 170, 3)


In [7]:
up2 = cv2.pyrUp(up)
cv_show(up2)
print(up2.shape)

(1768, 1360, 3)


In [12]:
up = cv2.pyrUp(img)
up_down = cv2.pyrDown(up)

res = np.hstack((img, up_down))
cv_show(res)
print(up_down.shape)

(442, 340, 3)


In [13]:
up = cv2.pyrUp(img)
up_down = cv2.pyrDown(up)
cv_show(img - up_down)

### 拉普拉斯金字塔
$L{_i} = G{_i} - PyrUp(PyrDown(G{_i}))$ <br>


In [14]:
down = cv2.pyrDown(img)
down_up = cv2.pyrUp(down)
L_1 = img - down_up
cv_show(L_1)

## 圖像輪廓

#### cv2.findContours(img, mode, method)
mode: 輪廓檢索模式
    * RETR_EXTERNAL: 指檢索最外面的輪廓
    * RETR_LIST: 檢索所有的輪廓, 並將其保存到一條LIST內
    * RETR_CCOMP: 檢索所有的輪廓, 並將他們組織為兩層, 頂層是個部份的外部邊界, 第二層是空洞的邊界
    * RETR_TREE: 檢索所有的輪廓, 並重構嵌套輪廓的整個層次 <----推薦
method: 輪廓逼近方式
    * CHAIN_APPROX_NONE: 以Freeman鍊碼的方式輸出輪廓, 所有其他方法輸出多邊形(頂點的序列)
    * CHAIN_APPROX_SIMPLE: 壓縮水平, 垂直和斜的部分, 也就是函數只保留他們的終點部分

In [19]:
img = cv2.imread('dataset/car.png') # read img
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # 轉灰階
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY) # 二值化
cv_show(thresh)

In [21]:
# binary, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)